In [25]:
from fastembed import TextEmbedding

In [26]:
# Initialize the embedding model
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [27]:
# Your query
query = "I just discovered the course. Can I join now?"

# Embed the text (returns a generator)
embedding = list(embedder.embed([query]))[0]  # Convert from generator and extract the single vector


In [28]:
import numpy as np

embedding = np.array(embedding)
print(embedding.shape)         # Should be (512,)
print(np.min(embedding))       # This gives the answer to the quiz


(512,)
-0.11726373885183883


In [30]:
np.linalg.norm(embedding)

1.0000000000000002

In [31]:
similarity = embedding.dot(embedding)

In [32]:
similarity

1.0000000000000004

In [34]:
doc = 'Can I still join the course after the start date?'

q1 = list(embedder.embed([query]))[0]
q2 = list(embedder.embed([doc]))[0]

similarity = q1.dot(q2)

print(similarity)


0.9008528895674548


In [35]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [36]:
# 1. Embed all document texts
doc_texts = [doc['text'] for doc in documents]
V = np.array(list(embedder.embed(doc_texts)))  # Shape: (5, 512)

# 2. Embed the query
query = "I just discovered the course. Can I join now?"
q = np.array(list(embedder.embed([query]))[0])  # Shape: (512,)

# 3. Compute cosine similarities
similarities = V.dot(q)

# 4. Find the index with the highest similarity
most_similar_index = int(np.argmax(similarities))
print("Most similar index:", most_similar_index)

Most similar index: 1


In [37]:
from fastembed import TextEmbedding
import numpy as np

embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Original documents
documents = [
    {'question': 'Course - Can I still join the course after the start date?',
     'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."},
    
    {'question': 'Course - Can I follow the course after it finishes?',
     'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.'},
    
    {'question': 'Course - When will the course start?',
     'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00..."},
    
    {'question': 'Course - What can I do before the course starts?',
     'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account...'},
    
    {'question': 'How can we contribute to the course?',
     'text': 'Star the repo! Share it with friends if you find it useful ❣️...'}
]

# Step 1: Create full_text field
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Step 2: Embed all full_texts
V_full = np.array(list(embedder.embed(full_texts)))

# Step 3: Embed query again
query = "I just discovered the course. Can I join now?"
q = np.array(list(embedder.embed([query]))[0])

# Step 4: Compute cosine similarities
similarities = V_full.dot(q)

# Step 5: Find best match
best_index = int(np.argmax(similarities))
print("Best matching index:", best_index)


Best matching index: 0


In [38]:
from fastembed import TextEmbedding

embedder = TextEmbedding(model_name="BAAI/bge-small-en")

# Example
embedding = list(embedder.embed(["This is a test."]))[0]
print(len(embedding))  # Should print 384


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

384


In [39]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)


In [40]:
texts = [doc['question'] + ' ' + doc['text'] for doc in documents]


In [41]:
from fastembed import TextEmbedding
import numpy as np

embedder = TextEmbedding(model_name="BAAI/bge-small-en")

embeddings = list(embedder.embed(texts))  # 384-dimensional


In [42]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

client = QdrantClient(":memory:")  # Use in-memory or Qdrant Cloud

# Create a collection
client.recreate_collection(
    collection_name="ml_faq",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# Add points
points = [
    PointStruct(id=i, vector=embeddings[i], payload={"text": texts[i]})
    for i in range(len(texts))
]

client.upsert(collection_name="ml_faq", points=points)


/tmp/ipykernel_33095/2453493047.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [43]:
query = "I just discovered the course. Can I join now?"
query_vector = list(embedder.embed([query]))[0]

search_result = client.search(
    collection_name="ml_faq",
    query_vector=query_vector,
    limit=1
)

print(search_result[0].score)


0.8703173995018005


/tmp/ipykernel_33095/3698080907.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(
